In [1]:
using DelimitedFiles, HDF5, FileIO
using Plots, Rsvg, LaTeXStrings, AverageShiftedHistograms
using Chemfiles, JUMD
using LinearAlgebra, Statistics
home = "/home/pbarletta/labo/18/lbp/run"
rtdos = "/home/pbarletta/labo/18/lbp/rtdos_log"
pyplot()

Plots.PyPlotBackend()

### Lee modos, volumen original y  volúmenes NDD. Obtiene VGV

In [2]:
pdb = "4xcp"
# Leo modos
modes_4xcp, evals_4xcp = JUMD.read_ptraj_modes(
    joinpath(home, pdb,  "pca", string("full_modes_", pdb)))

# AAs
aa3 = convert(Int64, length(evals_4xcp))
aa = convert(Int64, aa3 / 3)

# Leo volumen de la traj
trj_vol = convert(Array{Float64, 1}, 
    readdlm(joinpath(home, pdb, "cavidad", string("vol_trj_", pdb)))[:, 3]);
nframes = length(trj_vol)

# Leo volúmene del average
vol_avg_4xcp = convert(Float64, readdlm(joinpath(home, pdb,  "ndd", string("vol_avg_", pdb)))[3])

# Leo volúmenes NDD
ndd_out_4xcp = convert(Array{Float64, 1},
    readdlm(joinpath(home, pdb,  "ndd", string("out_ndd_", pdb)))[2:end, 2])

# VGV
vgv_4xcp = (ndd_out_4xcp .- vol_avg_4xcp) .* evals_4xcp
vgv_4xcp = vgv_4xcp ./ norm(vgv_4xcp);

##### tempor_res = readdlm(joinpath(home, pdb, "cont", "res", "nn_series_lig_prt"))
##### trj_contactos_res = convert(Array{Int64, 2}, tempor_res[2:end, 2:end]);
##### header_contactos_res = convert(Array{String, 1}, tempor_res[1, 2:end]);
##### nctc_res = length(header_contactos_res)

tempor_atm = readdlm(joinpath(home, pdb, "cont", "atm", "nn_series_lig_prt"))
trj_contactos_atm = convert(Array{Int64, 2}, tempor_atm[2:end, 2:end]);
header_contactos_atm = convert(Array{String, 1}, tempor_atm[1, 2:end]);
nctc_atm = length(header_contactos_atm);

# Headers
fid = h5open(joinpath(home, pdb, "cont", "header_ctcto.h5"), "w")
write(fid, "headers", header_contactos_atm)
close(fid)

# Contactos
fid = h5open(joinpath(home, pdb, "cont", "ctcto.h5"), "w")
write(fid, "contactos", trj_contactos_atm)
close(fid)

In [192]:
header_contactos = h5read(joinpath(home, pdb, "cont", "header_ctcto.h5"), "headers")
trj_contactos = h5read(joinpath(home, pdb, "cont", "ctcto.h5"), "contactos");
nctc = length(header_contactos)

header_contactos_prt_res = Array{Int64, 1}(undef, nctc)
header_contactos_prtAtm = Array{String, 1}(undef, nctc)
header_contactos_ligAtm = Array{String, 1}(undef, nctc);

for i in eachindex(header_contactos)
    tmp = split(header_contactos[i], "@")

    header_contactos_prt_res[i] = parse(Int64, tmp[1][2:end])
    header_contactos_prtAtm[i] = convert(String, split(tmp[2], "_")[1])
    header_contactos_ligAtm[i] = convert(String, tmp[end])
end

In [194]:
prt_res = Array{Int64, 1}(undef, 0)

push!(prt_res, header_contactos_prt_res[1])

1-element Array{Int64,1}:
 10

In [195]:
prt_res

1-element Array{Int64,1}:
 10

In [191]:
header_contactos_prtRes

4940-element Array{Int64,1}:
  10
  10
  10
  10
  10
  10
  10
  10
  13
  13
  14
  14
  14
   ⋮
  17
 136
 138
 138
 137
 139
 138
 138
 138
 136
  15
 136